In [63]:
from searchPlus import *
from copy import deepcopy

linha1= "  ##### \n"
linha2= "###...# \n"
linha3= "#o@$..# \n"
linha4= "###.$o# \n"
linha5= "#o##..# \n"
linha6= "#.#...##\n"
linha7= "#$.....#\n"
linha8= "#......#\n"
linha9= "########\n"
mundoStandard=linha1+linha2+linha3+linha4+linha5+linha6+linha7+linha8+linha9

class SokobanState(object):

    def __init__(self, sokoban, boxes):
        self.sokoban = tuple(sokoban)
        self.boxes = set(boxes)

    def __eq__(self, value: object) -> bool:
        return (self.sokoban == value.sokoban and
                self.boxes == value.boxes)


class Sokoban(Problem):

    directions = {
            'N': (-1, 0),  # Norte (acima)
            'W': (0, -1),  # Oeste (esquerda)
            'E': (0, 1),   # Leste (direita)
            'S': (1, 0),   # Sul (abaixo)
        }

    def __init__(self, situacaoInicial=mundoStandard):
        """Construtor de Sokoban"""
        sokoban, boxes, goals, corners = None, set(), set(), set()
        self.map = situacaoInicial.split('\n')[:-1]
        for i, line in enumerate(self.map):
            for j, char in enumerate(line):
                if char in "@+":
                    sokoban = (i, j)
                if char in "$*":
                    boxes.add((i, j))
                if char in "+*o":
                    goals.add((i, j))
                if char in ".@" and self.is_corner(i, j):
                    corners.add((i, j))
        self.corners = corners
        self.initial = SokobanState(sokoban, boxes)
        self.goals = set(goals)

    def is_corner(self, i, j):
        """Identifica as cantos"""
        if self.map[i - 1][j] == "#" and self.map[i][j - 1] == "#": # Canto superior esquerdo
            return True
        if self.map[i - 1][j] == "#" and self.map[i][j + 1] == "#": # Canto superior direito
            return True
        if self.map[i + 1][j] == "#" and self.map[i][j - 1] == "#": # Canto inferior esquerdo
            return True
        if self.map[i + 1][j] == "#" and self.map[i][j + 1] == "#": # Canto inferior direito
            return True
        return False

    def movimento_valido(self, state, x, y, dx, dy):
        """Verifica se o movimento na direção dx, dy é válido"""
        mapa = self.map
        comp = len(mapa)
        lag = len(mapa[0])

        # Posição para onde o Sokoban deseja ir
        new_x, new_y = x + dx, y + dy

        # Se a nova posição for navegável, o movimento é válido
        if mapa[new_x][new_y] != "#" and (new_x, new_y) not in state.boxes:
            return True

        # Se a nova posição contiver uma caixa, verifica se ela pode ser empurrada
        if (new_x, new_y) in state.boxes:
            proximo_x, proximo_y = new_x + dx, new_y + dy

            # Verifica se a posição seguinte é válida
            if proximo_x < 0 or proximo_x >= comp or proximo_y < 0 or proximo_y >= lag:
                return False

            # Verifica se a posição seguinte é uma caixa
            if (proximo_x, proximo_y) in state.boxes:
                return False

            proximo_destino = mapa[proximo_x][proximo_y]
            # A caixa pode ser empurrada se a próxima posição for navegável ou um objetivo
            if (proximo_x, proximo_y) in self.goals or (proximo_destino != "#" and (proximo_x, proximo_y) not in self.corners):
                return True

        # Qualquer outra situação torna o movimento inválido
        return False

    def actions(self, state):
        """Retorna uma lista de ações possíveis (N, S, W, E) para o Sokoban"""
        x, y = state.sokoban
        actions = []

        for direcao, (dx, dy) in self.directions.items():
            if self.movimento_valido(state, x, y, dx, dy):
                actions.append(direcao)
        return actions

    def result(self, state, action):
        """Devolve o estado que resulta de executar action em state"""
        nstate = SokobanState(deepcopy(state.sokoban), deepcopy(state.boxes))
        ni, nj = nstate.sokoban
        dx, dy = self.directions[action]
        nstate.sokoban = (ni + dx, nj + dy)

        if nstate.sokoban in nstate.boxes:
            nni, nnj = nstate.sokoban
            nstate.boxes.remove(nstate.sokoban)
            nstate.boxes.add((nni + dx, nnj + dy))
        return nstate

    def goal_test(self, state):
        """Verifica se state é um estado final"""
        return state.boxes == self.goals

    def executa(self, state, actions):
        """Partindo de state, executa a sequência (lista) de acções (em actions) e devolve o último estado"""
        nstate = SokobanState(deepcopy(state.sokoban), deepcopy(state.boxes))
        for a in actions:
            nstate = self.result(nstate, a)
        return nstate

    def display(self, state):
        """Devolve a grelha em modo txt"""
        map = ""
        for i, linha in enumerate(self.map):
            for j, celula in enumerate(linha):
                if (i, j) == state.sokoban:
                    map += "+" if (i, j) in self.goals else "@"
                elif (i, j) in state.boxes:
                    map +=  "*" if (i, j) in self.goals else "$"
                elif (i, j) in self.goals:
                    map += "o"
                elif celula in "# ":
                    map += celula
                else:
                    map += "."
            map += '\n'
        return map

    def __eq__(self, value: object) -> bool:
        return super().__eq__(value)

In [62]:
gx=Sokoban()
resultado,vis = breadth_first_search_iia_count(gx)
if resultado:
    print("Solução Larg-prim (grafo) com custo", str(resultado.path_cost)+":")
    print(resultado.solution())
else:
    print('Sem Solução')
print("Visitados:",vis)

Solução Larg-prim (grafo) com custo 20:
['E', 'N', 'E', 'E', 'S', 'W', 'W', 'W', 'E', 'S', 'E', 'S', 'S', 'W', 'S', 'W', 'S', 'W', 'N', 'N']
Visitados: 3814


In [226]:
p1=Sokoban()
p2=Sokoban()
print(p1.initial==p2.initial)

True


In [227]:
l1= '  ######\n'
l2= '  #.oo@#\n'
l3= '  #.$$.#\n'
l4= '  ##.###\n'
l5= '   #.#\n'
l6= '   #.#\n'
l7= '####.#\n'
l8= '#....##\n'
l9= '#.#...#\n'
l10= '#...#.#\n'
l11='###...#\n'
l12='  #####\n'

mod6=l1+l2+l3+l4+l5+l6+l7+l8+l9+l10+l11+l12
short=Sokoban(situacaoInicial=mod6)
short.display(short.initial)
ei=Sokoban()
print(short.initial==ei.initial)

False


In [67]:
g = Sokoban()
print(g.actions(g.initial))

['W', 'E']


In [68]:
l1= '  ######\n'
l2= '  #.oo@#\n'
l3= '  #.$$.#\n'
l4= '  ##.###\n'
l5= '   #.#\n'
l6= '   #.#\n'
l7= '####.#\n'
l8= '#....##\n'
l9= '#.#...#\n'
l10= '#...#.#\n'
l11='###...#\n'
l12='  #####\n'

mod6=l1+l2+l3+l4+l5+l6+l7+l8+l9+l10+l11+l12
short=Sokoban(situacaoInicial=mod6)
print(short.actions(short.initial))

['S', 'W']


In [69]:
linha1="##########\n"
linha2="#........#\n"
linha3="#..$..+..#\n"
linha4="#.....o..#\n"
linha5="##########\n"
mundoS=linha1+linha2+linha3+linha4+linha5
short=Sokoban(situacaoInicial=mundoS)
print(short.actions(short.initial))

[]


In [70]:
linha1="##########\n"
linha2="#.....$.##\n"
linha3="#ooo#$@$o#\n"
linha4="#o....$..#\n"
linha5="##########\n"
mundoS=linha1+linha2+linha3+linha4+linha5
short2=Sokoban(situacaoInicial=mundoS)
print(short2.actions(short2.initial))

['N', 'S', 'W', 'E']


In [71]:
inversa={'N': 'S', 'S': 'N','W': 'E', 'E': 'W'}
standard=Sokoban()
accoes=standard.actions(standard.initial)
#print(standard.initial)
for x in accoes:
    seg=standard.result(standard.initial,x)
    ni=standard.result(seg,inversa[x])
    print(standard.initial==ni)

False
False


In [72]:
g=Sokoban()
print(g.display(g.result(g.initial,g.actions(g.initial)[1])))

  ##### 
###...# 
#o @..# 
###.$o# 
#o##..# 
#.#...##
#$.....#
#......#
########



In [73]:
standard=Sokoban()
accoes=standard.actions(standard.initial)
ss=Sokoban()
for a in accoes:
    n=standard.result(standard.initial,a)
    print(standard.initial==ss.initial)

True
True


In [74]:
g=Sokoban()
print('Goal?',g.goal_test(g.initial))

Goal? False


In [75]:
linha1="##########\n"
linha2="#........#\n"
linha3="#..$..+..#\n"
linha4="#........#\n"
linha5="##########\n"
mundoS=linha1+linha2+linha3+linha4+linha5
short=Sokoban(situacaoInicial=mundoS)
seq=['W','W','N','W','W','S','E','E','E']
final=short.executa(short.initial,seq)
print(short.display(final))
print('Goal?',short.goal_test(final))

##########
#........#
#..$..+..#
#........#
##########

Goal? False


In [76]:
gx=Sokoban()
resultado,vis = breadth_first_search_iia_count(gx)
if resultado:
    print("Solução Larg-prim (grafo) com custo", str(resultado.path_cost)+":")
    print(resultado.solution())
else:
    print('Sem Solução')
print("Visitados:",vis)

NameError: name 'breadth_first_search_iia_count' is not defined